In [1]:
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS, XSD, DCTERMS, VOID, OWL
from rdflib.namespace import Namespace
import json
import urllib.parse

MCRO = Namespace("http://purl.obolibrary.org/obo/mcro.owl#")
DUL = Namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")

def load_triples(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def create_ttl(triples):
    g = Graph(identifier=MCRO.ModelCardGraph)
    
    # Bind namespaces
    g.bind("mcro", MCRO)
    g.bind("rdf", RDF)
    g.bind("rdfs", RDFS)
    g.bind("owl", OWL)
    g.bind("xsd", XSD)
    g.bind("dul", DUL)
    g.bind("dct", DCTERMS)
    g.bind("void", VOID)

    # Process class hierarchy
    classes = {}
    for triple in triples:
        s, p, o = triple['s'], triple['p'], triple['o']
        if p == 'rdf:type' and o.startswith(str(MCRO)):
            cls_uri = process_subject(s)
            classes[cls_uri] = {'type': 'Class'}
        elif p == 'rdfs:subClassOf':
            subj_uri = process_subject(s)
            super_uri = process_object(o, g)
            
            if subj_uri not in classes:
                classes[subj_uri] = {}
            classes[subj_uri]['super'] = super_uri

    # Add class declarations with hierarchy
    for cls_uri, data in classes.items():
        g.add((cls_uri, RDF.type, RDFS.Class))
        
        if cls_uri == MCRO.Component:
            g.add((cls_uri, RDFS.subClassOf, OWL.Thing))
        elif cls_uri == MCRO.Model:
            g.add((cls_uri, RDFS.subClassOf, MCRO.Component))
        else:
            if 'super' in data:
                g.add((cls_uri, RDFS.subClassOf, data['super']))
            else:
                g.add((cls_uri, RDFS.subClassOf, MCRO.Model))

    # Process all triples with semantic enhancements
    for triple in triples:
        s = process_subject(triple['s'])
        p = process_predicate(triple['p'])
        o = process_object(triple['o'], g)
        
        # Handle special cases for semantic enrichment
        if p == DUL.hasParameterDataValue:
            # Convert generic parameters to specific properties
            if "name" in str(s).lower():
                p = MCRO.hasName
            elif "accuracy" in str(s).lower():
                p = MCRO.hasAccuracy
            elif "precision" in str(s).lower():
                p = MCRO.hasPrecision
            elif "recall" in str(s).lower():
                p = MCRO.hasRecall
            elif "f1score" in str(s).lower():
                p = MCRO.hasF1Score
                
        # Add structured metrics
        if isinstance(o, Literal) and isinstance(o.value, str) and o.value.startswith('{'):
            # Handle JSON metrics by creating blank nodes
            metric_node = BNode()
            g.add((s, p, metric_node))
            g.add((metric_node, RDF.type, MCRO.Metric))
            
            data = json.loads(o.value)
            for key, values in data.items():
                category_node = BNode()
                g.add((metric_node, MCRO.hasCategory, category_node))
                g.add((category_node, MCRO.categoryName, Literal(key)))
                
                for k, v in values.items():
                    pred = getattr(MCRO, f"has{k.capitalize()}", None)
                    if pred:
                        g.add((category_node, pred, Literal(v)))
            continue
        
        # Add units where applicable
        if isinstance(o, Literal) and o.datatype == XSD.float:
            unit_pred = getattr(MCRO, f"{p.split('#')[-1]}Unit", None)
            if unit_pred:
                g.add((s, unit_pred, Literal("percent")))
        
        if isinstance(o, list):
            for obj in o:
                g.add((s, p, obj))
        else:
            g.add((s, p, o))

    # Validate syntax
    try:
        g.parse(data=g.serialize(format='turtle'), format='turtle')
    except Exception as e:
        print(f"Validation error: {e}")
        raise

    return g

def process_subject(subject_str):
    if subject_str.startswith(str(MCRO)):
        local_part = subject_str.split('#')[-1]
        return MCRO[local_part.replace(' ', '_')]
    elif subject_str.startswith('http'):
        return URIRef(subject_str)
    else:
        return BNode(subject_str)

def process_predicate(predicate_str):
    if predicate_str.startswith(str(MCRO)):
        local_part = predicate_str.split('#')[-1]
        return MCRO[local_part.replace(' ', '_')]
    
    if ':' in predicate_str:
        prefix, local = predicate_str.split(':', 1)
        prefix = prefix.lower()
        if prefix == 'rdf':
            return getattr(RDF, local)
        elif prefix == 'rdfs':
            return getattr(RDFS, local)
        elif prefix == 'xsd':
            return getattr(XSD, local)
        elif prefix == 'dul':
            return DUL[local]
        elif prefix == 'owl':
            return getattr(OWL, local)
        elif prefix == 'mcro':
            return MCRO[local.replace(' ', '_')]
        else:
            return URIRef(predicate_str)
    else:
        return MCRO[predicate_str.replace(' ', '_')]

def process_object(obj_str, graph):
    if isinstance(obj_str, str) and obj_str.startswith('{'):
        return Literal(obj_str, datatype=XSD.json)
    
    if isinstance(obj_str, str) and '^^' in obj_str:
        value, dtype = obj_str.rsplit('^^', 1)
        value = value.strip('"')
        return Literal(value, datatype=URIRef(dtype.strip()))
    
    if obj_str.lower() in ('true', 'false'):
        return Literal(obj_str.lower() == 'true', datatype=XSD.boolean)
    
    if obj_str.replace('.', '', 1).isdigit():
        return Literal(float(obj_str) if '.' in obj_str else int(obj_str))
    
    if obj_str.startswith(str(MCRO)):
        local_part = obj_str.split('#')[-1]
        return MCRO[local_part.replace(' ', '_')]
    
    if obj_str.startswith('http'):
        return URIRef(obj_str)
    
    return Literal(obj_str)

if __name__ == "__main__":
    triples = load_triples("triples.json")
    graph = create_ttl(triples)
    graph.serialize(destination="model_card.ttl", format="turtle")

/scratch-node/20230112.2646381/ipykernel_491020/1174850883.py:152: UserWarning: Code: json is not defined in namespace XSD
  return Literal(obj_str, datatype=XSD.json)
